# Defining a simple multiobjective optimization problem

**Caveats:** 

- When defining evaluators, be very careful with variable names spaces and variable capturing. Avoid lambdas, when possible.
- The names supplied to `ScalarConstraint` matter! The length of the name list is used to determine the number of objectives.
- Constraint evaluators should always accept two arguments: the decision variables and objective vectors. Most of the time, the latter is not used.
- _To make your life easy, minimize always and do not be cheesy!_

Here, we will be defining a simple multiobjctive optimization problem with both function and box-constraints on the variables.

## Imports

We start with the relevant imports.

In [38]:
import numpy as np

from desdeo_problem import VectorObjective, Variable, ScalarConstraint, MOProblem

## Objectives

Let us define a problem with three objectives and two variables. We start by defining the objective function.

In [39]:
def objective(x: np.ndarray) -> np.ndarray:
    # ensure that x is two dimensional
    x = np.atleast_2d(x)
    
    # x[:, 0] refers to first variable values, x[:, 1] to second
    f_1 = -x[:, 0] + x[:, 1]
    f_2 = x[:, 0] - x[:, 1]
    f_3 = x[:, 0] * x[:, 1] + 2.0
    
    # return each objective vector on a single row
    return np.stack((f_1, f_2, f_3), axis=1)

In [40]:
objective([[1,2], [3,4]])

array([[ 1., -1.,  4.],
       [ 1., -1., 14.]])

In [41]:
objective([2,2])

array([[0., 0., 6.]])

In [47]:
# name, evaluator
# evaluator must return objective vectors for VectorObjective
# Obs! The names are used to deduce the number of objectives; make sure it is correct!
objectives = [VectorObjective(["f1", "f2", "f3"], objective)]

## Variables

Next, we formally  define the variables with box constraints.

In [48]:
# name, initial value, lower bound, upper bound
x_1 = Variable("x1", 0, -5.2, 2.4)
x_2 = Variable("x2", 1.0, -2.2, 1.8)

# variables must be stored in a list when passed to MOProblem
variables = [x_1, x_2]

## Constraint

Then, we define a function constraints. We begin by evaluating the constraint similar to objective, when we pass it to `ScalarConstraint` so that we can define `MOProblem` using it.

In [60]:
def constraint(x: np.ndarray, _) -> np.ndarray:
    # constraint functions exactly like an objective would, but in this case, it will
    # return a single scalar value for each decision variable vector
    x = np.atleast_2d(x)
    
    # x1 + x2 should be more than 3
    con_value = x[:, 0] + x[:, 1] - 3.0
    
    return con_value

# name, n of decision vars, n of objectives, evaluator
# constraint must be supplied as a list to MOProblem
constraints = [ScalarConstraint("con1", 2, 3, constraint)]

## MOProblem instantation

Now we are ready to define the multiobjective optimization problem. We define it as an instance of `MOProblem`.

In [61]:
# objectives, variables, constraints
moproblem = MOProblem(objectives, variables, constraints)

The `moproblem` object can be evaluated using the `.evaluate`-method.

In [62]:
moproblem.evaluate(np.atleast_2d([2,2]))

EvaluationResults(objectives=array([[0., 0., 6.]]), fitness=array([[0., 0., 6.]]), constraints=array([[1.]]), uncertainity=array([[nan, nan, nan]]))

In [66]:
result = moproblem.evaluate(np.array([[-1, 1], [2, 1], [1.1, 2.2]]))
print(f"Objectives:\n{result.objectives}")
print(f"Constraint values:\n{result.constraints}")

Objectives:
[[ 2.   -2.    1.  ]
 [-1.    1.    4.  ]
 [ 1.1  -1.1   4.42]]
Constraint values:
[[-3. ]
 [ 0. ]
 [ 0.3]]


# Scalarizing MOProblems

Here we see an example how MOProblems can be scalarized. We begin with the relevant imports.

In [112]:
from desdeo_tools.scalarization import Scalarizer, StomASF
from desdeo_tools.solver import ScalarMethod, ScalarMinimizer

- `Scalarizer` is used to scalarize an instance of `MOProblem`, i.e., by some achievement scalarizing functions like `StomASF` here.
- `ScalarMethod` is used to define minimization routines to minimize scalar valued functions scalarized by `Scalarizer`.
- `ScalarMinimizer` utilizes `ScalarMehthod` to solve the scalarized problem.

Scalarizing `moproblem` with a custom scalarizer:

In [113]:
def not_very_smart(fs: np.ndarray, encouraging_message: str = None) -> np.ndarray:
    fs = np.atleast_2d(fs)
    
    if encouraging_message is not None:
        print(encouraging_message)
    
    return np.sum(fs, axis=1)

In [114]:
not_very_smart([1,2,3])

array([6])

In [115]:
not_very_smart([[1,2,3], [3,4,5]], encouraging_message="Do not worry, you will learn to scalarize MO problems properly some day...")

Do not worry, you will learn to scalarize MO problems properly some day...


array([ 6, 12])

## Defining a custom scalarizer

In [116]:
# evaluator from decision variables to objective vectors, scalarizer, arguments to the first evaluator, arguments to the scalarizer function
custom_scalarizer = Scalarizer(lambda x: moproblem.evaluate(x).objectives, not_very_smart, evaluator_args=None, scalarizer_args={"encouraging_message": "Scalarized!"})

In [117]:
custom_scalarizer.evaluate([1,2])

Scalarized!


/home/kilo/.cache/pypoetry/virtualenvs/desdeo-tutorial-HDcz-zLO-py3.9/lib/python3.9/site-packages/desdeo_problem/problem/Problem.py:1149: UserWarning: Some decision variable values violate upper bounds
  warn("Some decision variable values violate upper bounds")


array([4.])

In [118]:
# the scalarizer is called with decision variable values
custom_scalarizer.evaluate(np.array(np.array([[1,1], [1.5, 1.5]])))

Scalarized!


array([3.  , 4.25])

## Defining a custom solver

Here we define a custom solver by wrapping scipy's minimization routine into utilizing ScalarMethod.

In [137]:
from scipy.optimize import minimize as scipy_minimize
from typing import Callable

In [138]:
def custom_minimizer(objectives: Callable, x0: np.ndarray, bounds: np.ndarray, constraints: Callable, scipy_method: str = "SLSQP") -> dict:
    res = scipy_minimize(objectives, x0, bounds=bounds, constraints=constraints, method=scipy_method)
    return {"x": res.x, "success": res.success}

scipy_method = ScalarMethod(custom_minimizer, method_args={"scipy_method": "SLSQP"}, use_scipy=True)
custom_minimizer = ScalarMinimizer(custom_scalarizer, moproblem.get_variable_bounds(), constraint_evaluator=lambda x: moproblem.evaluate(x).constraints.squeeze(), method=scipy_method)

In [139]:
result = custom_minimizer.minimize(np.array([1,1]))

print(f"Optimization was successful? -> {result['success']}")
print(f"Solution:{result['x']}")
print(f"Objective values: {moproblem.evaluate(result['x']).objectives}")

Scalarized!
Scalarized!
Scalarized!
Scalarized!
Scalarized!
Scalarized!
Optimization was successful? -> True
Solution:[1.5 1.5]
Objective values: [[0.   0.   4.25]]


## Utilizing presets

In [209]:
ideal = np.array([-1.8, -0.6, 7])
asf = StomASF(ideal)

asf_scalarizer = Scalarizer(lambda x: moproblem.evaluate(x).objectives, asf, scalarizer_args={"reference_point": ideal})
preset_minimizer = ScalarMinimizer(asf_scalarizer, moproblem.get_variable_bounds(), constraint_evaluator=lambda x: moproblem.evaluate(x).constraints.squeeze(), method='scipy_de')

In [210]:
result = preset_minimizer.minimize(np.array([1,1]))

/home/kilo/.cache/pypoetry/virtualenvs/desdeo-tutorial-HDcz-zLO-py3.9/lib/python3.9/site-packages/desdeo_problem/problem/Problem.py:1149: UserWarning:

Some decision variable values violate upper bounds

/home/kilo/.cache/pypoetry/virtualenvs/desdeo-tutorial-HDcz-zLO-py3.9/lib/python3.9/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning:

delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.



In [211]:
print(f"Optimization was successful? -> {result['success']}")
print(f"Solution:{result['x']}")
print(f"Objective values: {moproblem.evaluate(result['x']).objectives}")

Optimization was successful? -> True
Solution:[2.28355206 1.68355206]
Objective values: [[-0.6         0.6         5.84447877]]


# Evolving MOProblems

We can find a bunch of approximate solutions utilizing evolutionary methods. We begin with the needed imports.

In [212]:
from desdeo_emo.EAs import NSGAIII

In [218]:
evolver = NSGAIII(moproblem,
                  interact=False,
                  n_iterations=10,
                  n_gen_per_iter=100,
                  population_size=100)

In [219]:
while evolver.continue_evolution():
    evolver.iterate()

In [220]:
# return the population and approximate Pareto front, .end() does non-dominated sorting
population, aprx_pf = evolver.end()

In [221]:
# clean up the population
mask = (moproblem.evaluate(population).constraints >= 0).flatten()

clean_pop = population[mask]
clean_frt = aprx_pf[mask]

In [223]:
# We will need an ideal and nadir later.
emo_ideal = np.min(clean_frt, axis=0)
emo_nadir = np.max(clean_frt, axis=0)

print(f"True ideal: {ideal}")
print(f"EMO ideal: {emo_ideal}")
print(f"EMO nadir: {emo_nadir}")

True ideal: [-1.8 -0.6  7. ]
EMO ideal: [-1.58347679  0.24996009  3.69208732]
EMO nadir: [-0.24996009  1.58347679  5.63986062]


In [226]:
# Ehhh, close enough...
moproblem.ideal = ideal
moproblem.nadir = emo_nadir

# Interacting with interactive methods

We will see how to interact with interactive methods by utilizing the reference point method in the `desdeo-mcdm` module. We begin by importing the method.

In [229]:
from desdeo_mcdm.interactive import ReferencePointMethod

Initializing the method:

In [230]:
rpm_method = ReferencePointMethod(moproblem, moproblem.ideal, moproblem.nadir)

Interactive methods are started by invoking the `start` method. The `start`-method will return our first `request`:

In [231]:
first_request = rpm_method.start()

Let us inspect the first request:

In [232]:
first_request.content

{'message': "Please specify a reference point as 'reference_point'.",
 'ideal': array([-1.8, -0.6,  7. ]),
 'nadir': array([-0.24996009,  1.58347679,  5.63986062])}

In [235]:
# The response is empty for now
first_request.response

The message will give us a tip how to proceed:

In [237]:
first_request.content["message"]

"Please specify a reference point as 'reference_point'."

Let us provide an initial referencep point as instructed. The reference point should be between the ideal and nadir points:


In [240]:
print(f"Ideal point: {first_request.content['ideal']}")
print(f"Nadir point: {first_request.content['nadir']}")

Ideal point: [-1.8 -0.6  7. ]
Nadir point: [-0.24996009  1.58347679  5.63986062]


In [241]:
reference_point = np.array([-1.5, 1.2, 6.1])

The reference point must be defined as part of the `response` to `first_request`:

In [242]:
our_response = {"reference_point": reference_point}
first_request.response = our_response

We may now continue iterating:


In [246]:
second_request = rpm_method.iterate(first_request)

# Obs! The reference point method does not care about constraints right now...
for key in second_request.content:
    print(f"{key}: {second_request.content[key]}\n")

message: In case you are satisfied with one of the solutions, please state:  1. 'satisfied' as 'True'.2. 'solution_index' as the index number of the solution you choose, so that first solution has index number of 0, second 1 and so on.Otherwise, please state 'satisfied' as 'False and specify a new reference point as 'reference_point'.

current_solution: [-4.59998191  4.59998191 -3.2799584 ]

additional_solutions: [array([-4.5999794,  4.5999794, -3.2799526]), array([-4.5999776 ,  4.5999776 , -3.27994847]), array([-4.59997562,  4.59997562, -3.27994391])]



Let us define a new reference point and iterate once more:

In [249]:
reference_point = np.array([-1.4, 1.4, 5.8])
second_request.response = {"reference_point": reference_point}

last_request = rpm_method.iterate(second_request)

In [250]:
# Obs! The reference point method does not care about constraints right now...
for key in last_request.content:
    print(f"{key}: {last_request.content[key]}\n")

message: In case you are satisfied with one of the solutions, please state:  1. 'satisfied' as 'True'.2. 'solution_index' as the index number of the solution you choose, so that first solution has index number of 0, second 1 and so on.Otherwise, please state 'satisfied' as 'False and specify a new reference point as 'reference_point'.

current_solution: [-4.59997248  4.59997248 -3.27993671]

additional_solutions: [array([-4.59997115,  4.59997115, -3.27993369]), array([-4.5999695,  4.5999695, -3.2799298]), array([-4.59996774,  4.59996774, -3.27992581])]



Suppose we are now satisfied with the second solution:

In [252]:
last_request.response = {"satisfied": True, "solution_index": 1}

end_request = rpm_method.iterate(last_request)

In [253]:
# Obs! The reference point method does not care about constraints right now...
for key in end_request.content:
    print(f"{key}: {end_request.content[key]}\n")

message: Final solution found.

solution: [ 2.39998537 -2.19998578]

objective_vector: [-4.59997115  4.59997115 -3.27993369]



As we can see, the reference point method has miserably failed to take into account the constraints. Why? Because at least I have tested most of the methods with problems without constraints. Lesson learned: test coverage matters!